In [59]:
import numpy as np
import time as ti
import scipy.sparse as spa
import scipy.sparse.linalg as las

In [60]:
def fun_f_a(x,n):
  f = np.sum([100*(x[i+1]-x[i]**2)**2+(x[i]-1)**2 for i in range(0,n-1)])
  return f

def fun_g_a(x,n):
  g = np.zeros(n)
  g[1:] = -200*np.power(x[:-1],2)
  g[:-1]+=-400*x[:-1]*(-np.power(x[:-1],2)+x[1:])
  g[1:-1]+=202*x[1:-1]
  g[0]+=2*x[0]
  g[-1]+=200*x[-1]
  g[:-1]+=-2
  return g

def fun_H_a(x,n):
  ds = np.array([-400*x[i] for i in range(0,n-1)])
  dp = np.zeros(n)
  dp[:-1] = np.array([1200*x[i]**2-400*x[i+1] for i in range(0,n-1)])
  dp[-1] = 200
  dp[1:-1]+=202
  dp[0]+=2
  ind = np.arange(0,n)
  I = np.concatenate((ind,ind[:-1],ind[:-1]+1))
  J = np.concatenate((ind,ind[:-1]+1,ind[:-1]))
  V = np.concatenate((dp,ds,ds))
  H = spa.coo_matrix((V,(I,J)))
  return H

In [61]:
def alpha_estimate_spa(xk,d_k,M):
  a = 0; b = 1; n = len(xk)
  for k in range(0,M):
    alp = np.linspace(a,b,num=5,endpoint=True)
    alp1 = alp[1]
    alp2 = alp[2]
    alp3 = alp[3]
    xk1 = xk+alp1*d_k
    xk2 = xk+alp2*d_k
    xk3 = xk+alp3*d_k
    f1 = fun_f_a(xk1,n)
    f2 = fun_f_a(xk2,n)
    f3 = fun_f_a(xk3,n)
    pos = np.argmin([f1,f2,f3])+1
    a = alp[pos-1]
    b = alp[pos+1]
  return (a+b)/2

In [62]:
def newton_optimization(n, initial_point, max_iter=50):
    xk = initial_point.copy()
    times_k = []
    norm_gk = []
    
    for k in range(max_iter):
        Hk = fun_H_a(xk, n)  # Calcula el Hessiano en el punto xk
        gk = fun_g_a(xk, n)  # Calcula el gradiente en el punto xk
        norm_gk.append(np.linalg.norm(gk))
        
        start_time = ti.time()
        dxk = las.spsolve(Hk, -gk)  # Resuelve el sistema lineal
        end_time = ti.time()
        
        times_k.append(end_time - start_time)  # Guarda el tiempo de ejecución para esta iteración
        
        alpha_opt = alpha_estimate_spa(xk, dxk, 20)  # Estima el mejor valor de alpha
        xk = xk + alpha_opt * dxk  # Actualiza el punto actual
    
    return times_k, norm_gk  # Devuelve la solución, los tiempos de cada iteración, y las normas del gradiente

In [63]:
# Ejemplo de uso:
n = 10**3
initial_point = 5 * np.ones(n)
times, gradients = newton_optimization(n, initial_point)

print("Execution times:", times)
print("Gradient norms:", gradients)

Execution times: [0.0, 0.0010113716125488281, 0.0, 0.0, 0.0010008811950683594, 0.0009999275207519531, 0.001001119613647461, 0.0, 0.0, 0.0010068416595458984, 0.0010089874267578125, 0.001005411148071289, 0.001005411148071289, 0.0, 0.0, 0.001001119613647461, 0.0010073184967041016, 0.0, 0.0, 0.0, 0.0, 0.0009996891021728516, 0.0, 0.0010046958923339844, 0.0010123252868652344, 0.0, 0.0010066032409667969, 0.0, 0.0, 0.001007080078125, 0.0, 0.0, 0.0010008811950683594, 0.0, 0.0010051727294921875, 0.0, 0.0, 0.0010008811950683594, 0.0010006427764892578, 0.0010089874267578125, 0.0, 0.0, 0.0010044574737548828, 0.0010213851928710938, 0.001001119613647461, 0.0, 0.0, 0.0, 0.0, 0.0]
Gradient norms: [1138244.0651881301, 336020.93500914285, 278964.09310751426, 28787.15841720946, 57695.23318264561, 3021.0429103776737, 7875.2016690855435, 495.4779510019921, 1280.482323341318, 53.22391251838089, 237.62243663517472, 114.71375040918196, 199.80434771663153, 93.2157286999676, 165.68580722112523, 78.03533468592511